# 🎡 Wheel Strategy Backtester V2 - REAL DATA

This notebook implements the Wheel Strategy using **real market data from Databento**.

## Strategy Summary

"On fundamentally strong stocks, sell **30-delta puts** (30–45 DTE) only when **price ≤ 20-day SMA** or **≤ lower Bollinger Band** (20,2); if assigned, sell **30-delta calls** (30–45 DTE) until called away; close positions at **50% profit** or **21 DTE remaining**."

## Architecture

- **Function-based**: All code uses functions (no classes)
- **Self-contained**: All code, API keys, and logic in this notebook
- **UV package management**: Uses UV instead of pip/conda
- **Disk caching**: Data cached as parquet files for efficiency
- **Inspection-friendly**: All major functions output DataFrames/dicts for user inspection

## Data Flow

```
┌─────────────────────────────────────────────────────────────┐
│ SECTION 1: Configuration & Setup                           │
│ - Imports, API keys, strategy parameters, cache config      │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 2: Data Fetching Functions                          │
│ - Databento client, equity data, options data, market data  │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 3: Data Caching Functions                          │
│ - Cache paths, read/write, statistics, loading              │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 4: Data Processing Functions                       │
│ - Delta calculation, technical indicators, beta             │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 5: Filtering Functions                             │
│ - Fundamental filters, technical filters, chain filters    │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 6: Strategy Logic Functions                        │
│ - Position management, contract selection, rolling logic  │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 7: Backtest Engine Functions                       │
│ - Portfolio state, trade execution, daily processing       │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 8: Reporting Functions                             │
│ - Metrics calculation, visualization, reporting            │
└──────────────────────┬──────────────────────────────────────┘
                       │
┌──────────────────────▼──────────────────────────────────────┐
│ SECTION 9: Main Execution                                 │
│ - Cache inspection, data collection, pre-computation,      │
│   backtest execution, results & visualization              │
└─────────────────────────────────────────────────────────────┘
```

## Section Breakdown

1. **Section 0**: Overview & Data Flow (this cell)
2. **Section 1**: Configuration & Setup
3. **Section 2**: Data Fetching Functions
4. **Section 3**: Data Caching Functions
5. **Section 4**: Data Processing Functions
6. **Section 5**: Filtering Functions
7. **Section 6**: Strategy Logic Functions
8. **Section 7**: Backtest Engine Functions
9. **Section 8**: Reporting Functions
10. **Section 9**: Main Execution


---

# SECTION 1: Configuration & Setup

## 1.1 Package Installation (UV)

**Data Flow:**
- **Input**: None
- **Processing**: Install required packages using UV
- **Output**: Packages installed and ready
- **Dependencies**: None

**Note**: Run this cell first to install all dependencies using UV.



In [ ]:
# ============================================================================
# INSTALL PACKAGES USING UV
# ============================================================================
# Run this cell first to install all required packages

# List of required packages
required_packages = [
    "pandas",
    "numpy", 
    "databento",
    "yfinance",
    "scipy",
    "matplotlib",
    "seaborn",
    "tqdm",
    "python-dotenv"
]

# Check which packages are missing
print("Checking required packages...")
print("=" * 60)

missing = []
for pkg in required_packages:
    try:
        __import__(pkg)
        print(f"✅ {pkg} already installed")
    except ImportError:
        print(f"⚠️  {pkg} not found")
        missing.append(pkg)

print("=" * 60)

# Install missing packages using UV
if missing:
    print(f"Installing {len(missing)} missing packages using UV...")
    packages_str = " ".join(missing)
    !uv pip install {packages_str}
    print(f"✅ {packages_str} Installation complete!")
else:
    print("✅ All packages ready!")


Checking required packages...
✅ pandas already installed
✅ numpy already installed
✅ databento already installed
✅ yfinance already installed
✅ scipy already installed
✅ matplotlib already installed
⚠️  seaborn not found
✅ tqdm already installed
Installing 1 missing packages using UV...
Using Python 3.13.5 environment at: /Users/samuelminer/Projects/nissan_options/wheel_strategy/.venv
Audited 1 package in 2ms
✅ seaborn Installation complete!


## 1.2 API Configuration

**Data Flow:**
- **Input**: Environment variables from `.env` file
- **Processing**: Load API keys from environment variables and set up configuration
- **Output**: API client and configuration variables
- **Dependencies**: Section 1.1 (imports)

**Note**: The API key is loaded from the `DATABENTO_API_KEY` environment variable, which should be set in the `.env` file in the project root directory. This ensures the key is never committed to version control.


In [ ]:
# Load API key from environment variables (stored in .env file)
from pathlib import Path
from dotenv import load_dotenv
import os

# Load environment variables from .env file in project root
# Try multiple possible locations for the .env file
possible_env_paths = [
    Path.cwd().parent / ".env",  # Parent directory (project root)
    Path.cwd() / ".env",          # Current directory
    Path.home() / ".env",         # Home directory (fallback)
]

env_path = None
for path in possible_env_paths:
    if path.exists():
        env_path = path
        break

if env_path:
    load_dotenv(env_path, override=True)
    print(f"   Loaded .env from: {env_path}")
else:
    # Try loading from current directory anyway (might be set as env var)
    load_dotenv(override=False)

# Get API key from environment variable
DATABENTO_API_KEY = os.getenv("DATABENTO_API_KEY")
if not DATABENTO_API_KEY:
    raise ValueError(
        "DATABENTO_API_KEY not found in environment variables. "
        "Please set it in your .env file (in the project root) or as an environment variable."
    )

# Dataset IDs
OPRA_DATASET = "OPRA.PILLAR"  # Options data
XNAS_DATASET = "XNAS.ITCH"    # Equity data

print("✅ API Configuration loaded")


✅ API Configuration loaded
   Equity time: 15:30:00
   Options time: 15:45:00


## 1.3 Strategy Parameters

**Data Flow:**
- **Input**: None
- **Processing**: Define all strategy parameters as simple variables/dicts
- **Output**: Configuration variables available for use
- **Dependencies**: None


In [ ]:
# Data Fetching Times (hardcoded in strategy parameters)
EQUITY_TIME = "15:30:00"  # 3:30 PM for equity prices (SMA/BB calculation)
OPTIONS_TIME = "15:45:00"  # 3:45 PM for options execution

# Put Selling Parameters
PUT_DELTA_TARGET = 0.30
PUT_DELTA_BAND = 0.05  # ±0.05, so range is 0.25-0.35
PUT_DTE_MIN = 30
PUT_DTE_MAX = 45
PUT_DTE_TARGET = 35
PUT_MIN_PREMIUM_ROC = 0.02  # 2% minimum return on capital

# Call Selling Parameters (Covered Calls)
CALL_DELTA_DEFAULT = 0.30
CALL_DELTA_DEFENSIVE = 0.20  # When underwater
CALL_DELTA_AGGRESSIVE = 0.35  # When profitable + near upper BB
CALL_DELTA_AGGRESSIVE_MAX = 0.40
CALL_DTE_MIN = 30
CALL_DTE_MAX = 45

# Technical Indicators
SMA_WINDOW = 20
BB_WINDOW = 20
BB_STD_DEV = 2.0

# Calculation Windows
BETA_WINDOW_DAYS = 252 * 3  # 3 years of trading days for beta calculation
VOLATILITY_WINDOW_DAYS = 30  # 30 days for volatility estimation

# Fundamental Filters
MIN_MARKET_CAP = 10_000_000_000  # $10B
MAX_MARKET_CAP = None  # No upper limit
MIN_BETA = 0.3
MAX_BETA = 1.5
MIN_PRICE = 20.0
MAX_PRICE = 500.0

# Liquidity Filters
MIN_OPEN_INTEREST = 1000
MIN_VOLUME = 0  # No minimum volume requirement
MAX_SPREAD_PCT = 0.10  # 10% max bid-ask spread

# Position Management
PROFIT_TARGET_PCT = 0.50  # Close at 50% profit
MIN_DTE_TO_HOLD = 21  # Close if DTE < 21
MAX_POSITIONS = 10
INITIAL_CASH = 100_000.0

# Backtest Period
WARMUP_DAYS = 20  # Days needed for SMA/BB calculation # This should be rolling wing
START_DATE = date(2024, 1, 2)
END_DATE = date(2024, 12, 6)

# Technical Filter Type
# Options: "SMA_OR_BOLLINGER", "SMA_ONLY", "BOLLINGER_ONLY", "NONE"
TECHNICAL_FILTER_TYPE = "SMA_OR_BOLLINGER"

print("✅ Strategy Parameters loaded")
print(f"   Equity time: {EQUITY_TIME}")
print(f"   Options time: {OPTIONS_TIME}")
print(f"   Put Delta: {PUT_DELTA_TARGET} ± {PUT_DELTA_BAND}")
print(f"   Put DTE: {PUT_DTE_MIN}-{PUT_DTE_MAX}")
print(f"   Beta window: {BETA_WINDOW_DAYS} days ({BETA_WINDOW_DAYS/252:.1f} years)")
print(f"   Volatility window: {VOLATILITY_WINDOW_DAYS} days")
print(f"   Initial Cash: ${INITIAL_CASH:,.0f}")


✅ Strategy Parameters loaded
   Equity time: 15:30:00
   Options time: 15:45:00
   Put Delta: 0.3 ± 0.05
   Put DTE: 30-45
   Initial Cash: $100,000


## 1.4 Calculations & Formulas

**Data Flow:**
- **Input**: None
- **Processing**: Document all mathematical formulas used in the strategy
- **Output**: Formula definitions for reference
- **Dependencies**: None

This section documents the mathematical formulas used throughout the backtest.


In [ ]:
# ============================================================================
# CALCULATION FORMULAS
# ============================================================================
# This section documents all mathematical formulas used in the strategy

"""
OPTION DELTA (Black-Scholes Model)
-----------------------------------
Delta measures the sensitivity of option price to underlying price movement.

For Calls:  Δ = N(d1)
For Puts:   Δ = N(d1) - 1

Where:
    d1 = [ln(S/K) + (r + 0.5*σ²)*T] / (σ*√T)
    
    S = Current stock price (spot)
    K = Strike price
    r = Risk-free rate (default: 0.05 = 5%)
    σ = Annualized volatility (default: 0.25 = 25%)
    T = Time to expiration in years (DTE / 365)
    N() = Cumulative standard normal distribution

Delta ranges:
    - Calls: 0 to 1 (0 = OTM, 1 = deep ITM)
    - Puts: -1 to 0 (0 = OTM, -1 = deep ITM)
    - For filtering, we use absolute value for puts
"""

"""
BETA (Market Correlation)
--------------------------
Beta measures stock's sensitivity to market movements.

    β = Covariance(stock_returns, market_returns) / Variance(market_returns)

Where:
    stock_returns = Daily percentage returns of stock
    market_returns = Daily percentage returns of market index (e.g., SPY)
    
    Covariance = E[(stock - E[stock]) * (market - E[market])]
    Variance = E[(market - E[market])²]
    
    Window: BETA_WINDOW_DAYS (default: 756 days = 3 years)

Interpretation:
    β = 1.0: Moves with market
    β > 1.0: More volatile than market
    β < 1.0: Less volatile than market
    β = 0.0: No correlation with market
"""

"""
VOLATILITY ESTIMATION
---------------------
Annualized volatility from historical prices.

    σ_annual = σ_daily * √252

Where:
    σ_daily = Standard deviation of daily returns
    252 = Number of trading days per year
    Window: VOLATILITY_WINDOW_DAYS (default: 30 days)

Calculation:
    1. Calculate daily returns: r_t = (P_t - P_{t-1}) / P_{t-1}
    2. Calculate standard deviation of returns over VOLATILITY_WINDOW_DAYS
    3. Annualize by multiplying by √252
"""

"""
SIMPLE MOVING AVERAGE (SMA)
----------------------------
Average price over a lookback window.

    SMA_n = (P_t + P_{t-1} + ... + P_{t-n+1}) / n

Where:
    n = Window size (default: 20 days)
    P_t = Price at time t
"""

"""
BOLLINGER BANDS
---------------
Volatility-based price bands around SMA.

    Middle Band = SMA_n
    Upper Band  = SMA_n + (k * σ_n)
    Lower Band  = SMA_n - (k * σ_n)

Where:
    SMA_n = Simple moving average over n periods
    σ_n = Standard deviation of prices over n periods
    k = Number of standard deviations (default: 2.0)

Interpretation:
    - Price ≤ Lower Band: Oversold (potential put entry)
    - Price ≥ Upper Band: Overbought (potential call entry)
    - Price near Middle: Neutral
"""

"""
RETURN ON CAPITAL (ROC) - Premium
----------------------------------
Return on capital at risk for option premium.

    ROC = (Premium / Strike) * 100

Where:
    Premium = Option premium received
    Strike = Strike price (capital at risk for cash-secured puts)

Example:
    Premium = $2.00, Strike = $100
    ROC = (2.00 / 100) * 100 = 2.0%
"""

print("✅ Calculation Formulas Documented")
print("   Formulas: Delta, Beta, Volatility, SMA, Bollinger Bands, ROC")


✅ Calculation Formulas Documented
   Formulas: Delta, Beta, Volatility, SMA, Bollinger Bands, ROC


## 1.5 Cache Configuration

**Data Flow:**
- **Input**: None
- **Processing**: Set up cache directory paths and naming conventions
- **Output**: Cache configuration variables
- **Dependencies**: None


In [16]:
# Cache directory structure
NOTEBOOK_DIR = Path.cwd()
CACHE_BASE_DIR = NOTEBOOK_DIR / "data" / "cache"
CACHE_EQUITY_DIR = CACHE_BASE_DIR / "equity"
CACHE_OPTIONS_DIR = CACHE_BASE_DIR / "options"
CACHE_PROCESSED_DIR = CACHE_BASE_DIR / "processed"

# Create cache directories if they don't exist
CACHE_EQUITY_DIR.mkdir(parents=True, exist_ok=True)
CACHE_OPTIONS_DIR.mkdir(parents=True, exist_ok=True)
CACHE_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Cache Configuration loaded")
print(f"   Cache base: {CACHE_BASE_DIR}")
print(f"   Equity cache: {CACHE_EQUITY_DIR}")
print(f"   Options cache: {CACHE_OPTIONS_DIR}")
print(f"   Processed cache: {CACHE_PROCESSED_DIR}")


✅ Cache Configuration loaded
   Cache base: /Users/samuelminer/Projects/nissan_options/wheel_strategy/notebooks/data/cache
   Equity cache: /Users/samuelminer/Projects/nissan_options/wheel_strategy/notebooks/data/cache/equity
   Options cache: /Users/samuelminer/Projects/nissan_options/wheel_strategy/notebooks/data/cache/options
   Processed cache: /Users/samuelminer/Projects/nissan_options/wheel_strategy/notebooks/data/cache/processed


---

# SECTION 2: Data Fetching Functions

## 2.1 Databento Client Setup

**Data Flow:**
- **Input**: API key
- **Processing**: Create Databento client
- **Output**: Client object
- **Dependencies**: Section 1.1 (imports), Section 1.2 (API config)


In [17]:
def create_databento_client(api_key: str) -> db.Historical:
    """
    Create and return a Databento historical client.
    
    Args:
        api_key: Databento API key
        
    Returns:
        Databento historical client
    """
    return db.Historical(api_key)

# Create client
db_client = create_databento_client(DATABENTO_API_KEY)

# Output for inspection
print("✅ Databento Client Created")
print(f"   Client type: {type(db_client)}")
print(f"   API key set: {'Yes' if DATABENTO_API_KEY else 'No'}")


✅ Databento Client Created
   Client type: <class 'databento.historical.client.Historical'>
   API key set: Yes


## 2.2 Equity Data Fetching

**Data Flow:**
- **Input**: Symbols, start_date, end_date
- **Processing**: Fetch equity OHLC data at 3:30 PM (EQUITY_TIME) for each trading day
- **Output**: Dict[str, DataFrame] with columns: date, open, high, low, close, price
- **Dependencies**: Section 2.1 (Databento client), Section 1.3 (EQUITY_TIME parameter)


In [ ]:
def fetch_equity_data(
    symbols: List[str],
    start_date: date,
    end_date: date,
    client: db.Historical,
) -> Dict[str, pd.DataFrame]:
    """
    Fetch equity OHLC data at 3:30 PM for full backtest period.
    
    Fetches minute-level OHLC data at EQUITY_TIME (3:30 PM) for each trading day.
    Returns DataFrame with date, open, high, low, close, and price (close) columns.
    
    Args:
        symbols: List of stock symbols
        start_date: Start date (includes warmup period)
        end_date: End date (end of trading period)
        client: Databento client
        
    Returns:
        Dict[str, DataFrame] with columns: date, open, high, low, close, price
        DataFrame is sorted chronologically
    """
    result = {}
    
    # Generate list of trading dates
    current_date = start_date
    trading_dates = []
    while current_date <= end_date:
        # Skip weekends (simplified - in production, use trading calendar)
        if current_date.weekday() < 5:  # Monday = 0, Friday = 4
            trading_dates.append(current_date)
        current_date += timedelta(days=1)
    
    for symbol in tqdm(symbols, desc="Fetching equity data"):
        prices = []
        
        for trade_date in trading_dates:
            try:
                # Fetch OHLC data at EQUITY_TIME (3:30 PM)
                datetime_str = f"{trade_date.isoformat()}T{EQUITY_TIME}"
                data = client.timeseries.get_range(
                    dataset=XNAS_DATASET,
                    symbols=[symbol],
                    schema="ohlcv-1m",
                    start=datetime_str,
                    end=datetime_str,
                )
                
                if data is not None and len(data) > 0:
                    # Get the last row (in case there are multiple ticks at 3:30 PM)
                    row = data.iloc[-1]
                    prices.append({
                        'date': trade_date,
                        'open': float(row.get('open', 0)),
                        'high': float(row.get('high', 0)),
                        'low': float(row.get('low', 0)),
                        'close': float(row.get('close', 0)),
                        'price': float(row.get('close', 0)),  # Alias for convenience
                    })
            except Exception as e:
                # Skip dates with errors, continue to next date
                continue
        
        if prices:
            df = pd.DataFrame(prices)
            df = df.sort_values('date').reset_index(drop=True)
            result[symbol] = df
        else:
            result[symbol] = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'price'])
    
    # Output for inspection
    total_days = sum(len(df) for df in result.values())
    print(f"✅ Fetched equity data at {EQUITY_TIME} for {len(symbols)} symbols")
    print(f"   Total data points: {total_days}")
    print(f"   Date range: {start_date} to {end_date}")
    
    return result

print("✅ Equity Data Fetching Functions Defined")


✅ Equity Data Fetching Functions Defined


## 2.3 Options Data Fetching

**Data Flow:**
- **Input**: Symbol(s), date(s), time string, DTE range
- **Processing**: Fetch option definitions and prices from Databento
- **Output**: DataFrame with option chain data
- **Dependencies**: Section 2.1 (Databento client)


In [ ]:
def fetch_option_definitions(symbol: str, date: date, client: db.Historical) -> List[str]:
    """
    Fetch option definitions (contract symbols) for a given symbol and date.
    
    Args:
        symbol: Stock symbol (e.g., "AAPL")
        date: Trading date
        client: Databento client
        
    Returns:
        List of option contract symbols (e.g., ["AAPL 240119C00150000", ...])
    """
    try:
        # Use symbology.resolve to get option definitions
        definitions = client.symbology.resolve(
            dataset=OPRA_DATASET,
            symbols=[f"{symbol}.OPT"],
            stype_in="smart",
            stype_out="raw_symbol",
            start_date=date.isoformat(),
            end_date=date.isoformat(),
        )
        if definitions is not None and len(definitions) > 0:
            return definitions['raw_symbol'].tolist()
        return []
    except Exception as e:
        print(f"Error fetching definitions for {symbol} on {date}: {e}")
        return []


def fetch_options_prices(option_symbols: List[str], date: date, 
                        time_str: str, client: db.Historical) -> pd.DataFrame:
    """
    Fetch option prices at a specific time.
    
    Args:
        option_symbols: List of option contract symbols
        date: Trading date
        time_str: Time string (e.g., "15:45:00")
        client: Databento client
        
    Returns:
        DataFrame with columns: symbol, bid, ask, volume, open_interest, etc.
    """
    if not option_symbols:
        return pd.DataFrame()
    
    try:
        datetime_str = f"{date.isoformat()}T{time_str}"
        data = client.timeseries.get_range(
            dataset=OPRA_DATASET,
            symbols=option_symbols,
            schema="ohlcv-1m",
            start=datetime_str,
            end=datetime_str,
        )
        if data is not None and len(data) > 0:
            return data
        return pd.DataFrame()
    except Exception as e:
        print(f"Error fetching option prices on {date}: {e}")
        return pd.DataFrame()


def fetch_options_chain(symbol: str, date: date, time_str: str, 
                       min_dte: int, max_dte: int, client: db.Historical) -> pd.DataFrame:
    """
    Fetch full options chain with prices, filtered by DTE range.
    
    Args:
        symbol: Stock symbol
        date: Trading date
        time_str: Time string
        min_dte: Minimum days to expiration
        max_dte: Maximum days to expiration
        client: Databento client
        
    Returns:
        DataFrame with option chain data (columns: type, strike, expiration, dte, bid, ask, etc.)
    """
    # Fetch definitions
    option_symbols = fetch_option_definitions(symbol, date, client)
    if not option_symbols:
        return pd.DataFrame()
    
    # Fetch prices
    prices_df = fetch_options_prices(option_symbols, date, time_str, client)
    if prices_df.empty:
        return pd.DataFrame()
    
    # Parse option symbols to extract type, strike, expiration
    # Format: "AAPL 240119C00150000" -> type=C, strike=150, expiration=2024-01-19
    chain_data = []
    for opt_symbol in option_symbols:
        try:
            # Parse raw symbol (simplified - actual format may vary)
            parts = opt_symbol.split()
            if len(parts) >= 2:
                underlying = parts[0]
                contract = parts[1]
                
                # Extract expiration (first 6 digits: YYMMDD)
                exp_str = contract[:6]
                year = 2000 + int(exp_str[:2])
                month = int(exp_str[2:4])
                day = int(exp_str[4:6])
                expiration = date(year, month, day)
                
                # Calculate DTE
                dte = (expiration - date).days
                
                # Filter by DTE
                if min_dte <= dte <= max_dte:
                    # Extract type (C or P)
                    opt_type = contract[6] if len(contract) > 6 else None
                    if opt_type == 'C':
                        opt_type = 'call'
                    elif opt_type == 'P':
                        opt_type = 'put'
                    else:
                        continue
                    
                    # Extract strike (remaining digits, divide by 1000)
                    strike_str = contract[7:] if len(contract) > 7 else None
                    if strike_str:
                        strike = float(strike_str) / 1000.0
                    else:
                        continue
                    
                    # Get price data if available
                    price_row = prices_df[prices_df['symbol'] == opt_symbol] if 'symbol' in prices_df.columns else None
                    bid = float(price_row['bid'].iloc[0]) if price_row is not None and not price_row.empty and 'bid' in price_row.columns else None
                    ask = float(price_row['ask'].iloc[0]) if price_row is not None and not price_row.empty and 'ask' in price_row.columns else None
                    volume = int(price_row['volume'].iloc[0]) if price_row is not None and not price_row.empty and 'volume' in price_row.columns else 0
                    oi = int(price_row['open_interest'].iloc[0]) if price_row is not None and not price_row.empty and 'open_interest' in price_row.columns else 0
                    
                    chain_data.append({
                        'symbol': opt_symbol,
                        'underlying': underlying,
                        'type': opt_type,
                        'strike': strike,
                        'expiration': expiration,
                        'dte': dte,
                        'bid': bid,
                        'ask': ask,
                        'volume': volume,
                        'open_interest': oi,
                    })
        except Exception as e:
            continue
    
    if chain_data:
        chain_df = pd.DataFrame(chain_data)
        # Output for inspection
        print(f"✅ Fetched options chain for {symbol} on {date}")
        print(f"   Contracts: {len(chain_df)}")
        print(f"   DTE range: {chain_df['dte'].min()}-{chain_df['dte'].max()}")
        return chain_df
    return pd.DataFrame()

print("✅ Options Data Fetching Functions Defined")


In [ ]:
def fetch_market_cap(symbol: str) -> float:
    """
    Fetch market capitalization using yfinance.
    
    Args:
        symbol: Stock symbol
        
    Returns:
        Market cap in dollars, or None if not found
    """
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        market_cap = info.get('marketCap', None)
        return float(market_cap) if market_cap else None
    except Exception as e:
        print(f"Error fetching market cap for {symbol}: {e}")
        return None


def fetch_market_caps(symbols: List[str]) -> Dict[str, float]:
    """
    Fetch market caps for multiple symbols.
    
    Args:
        symbols: List of stock symbols
        
    Returns:
        Dict[str, float] with symbol -> market cap mapping
    """
    result = {}
    for symbol in tqdm(symbols, desc="Fetching market caps"):
        market_cap = fetch_market_cap(symbol)
        if market_cap:
            result[symbol] = market_cap
    return result


def fetch_index_prices(
    index_symbol: str,
    start_date: date,
    end_date: date,
    client: db.Historical,
) -> pd.DataFrame:
    """
    Fetch index prices at 3:30 PM for beta calculation.
    
    Uses the same approach as equity data fetching (OHLC at EQUITY_TIME).
    
    Args:
        index_symbol: Index symbol (e.g., "SPY" for S&P 500)
        start_date: Start date
        end_date: End date
        client: Databento client
        
    Returns:
        DataFrame with columns: date, price
    """
    # Generate list of trading dates
    current_date = start_date
    trading_dates = []
    while current_date <= end_date:
        if current_date.weekday() < 5:  # Skip weekends
            trading_dates.append(current_date)
        current_date += timedelta(days=1)
    
    prices = []
    for trade_date in tqdm(trading_dates, desc=f"Fetching {index_symbol}"):
        try:
            datetime_str = f"{trade_date.isoformat()}T{EQUITY_TIME}"
            data = client.timeseries.get_range(
                dataset=XNAS_DATASET,
                symbols=[index_symbol],
                schema="ohlcv-1m",
                start=datetime_str,
                end=datetime_str,
            )
            
            if data is not None and len(data) > 0:
                row = data.iloc[-1]
                prices.append({
                    'date': trade_date,
                    'price': float(row.get('close', 0)),
                })
        except Exception as e:
            continue
    
    if prices:
        df = pd.DataFrame(prices)
        df = df.sort_values('date').reset_index(drop=True)
        print(f"✅ Fetched {len(df)} days of {index_symbol} prices at {EQUITY_TIME}")
        print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
        return df
    else:
        return pd.DataFrame(columns=['date', 'price'])

print("✅ Market Data Fetching Functions Defined")


---

# SECTION 3: Data Caching Functions

## 3.1 Cache Path Management

**Data Flow:**
- **Input**: Data type, symbol, date, time string
- **Processing**: Generate cache file paths
- **Output**: Path object
- **Dependencies**: Section 1.4 (cache config)


In [ ]:
def get_cache_path(data_type: str, symbol: str, date: date, time_str: str = None) -> Path:
    """
    Generate cache file path for a given data type, symbol, and date.
    
    Args:
        data_type: "equity", "options", or "processed"
        symbol: Stock symbol
        date: Trading date
        time_str: Optional time string (for time-specific data)
        
    Returns:
        Path object for cache file
    """
    if data_type == "equity":
        base_dir = CACHE_EQUITY_DIR
        time_suffix = f"_{time_str.replace(':', '')}" if time_str else ""
        filename = f"{symbol}_{date.isoformat()}{time_suffix}.parquet"
    elif data_type == "options":
        base_dir = CACHE_OPTIONS_DIR
        time_suffix = f"_{time_str.replace(':', '')}" if time_str else ""
        filename = f"{symbol}_{date.isoformat()}{time_suffix}.parquet"
    elif data_type == "processed":
        base_dir = CACHE_PROCESSED_DIR
        filename = f"{symbol}_{date.isoformat()}.parquet"
    else:
        raise ValueError(f"Unknown data_type: {data_type}")
    
    return base_dir / filename


def ensure_cache_dir(cache_path: Path) -> None:
    """
    Ensure the directory for a cache path exists.
    
    Args:
        cache_path: Path to cache file
    """
    cache_path.parent.mkdir(parents=True, exist_ok=True)

print("✅ Cache Path Management Functions Defined")


## 3.2 Cache Read/Write

**Data Flow:**
- **Input**: Data, cache path
- **Processing**: Save/load data to/from parquet files
- **Output**: DataFrame or None
- **Dependencies**: Section 3.1 (cache paths)


In [ ]:
def load_cached_data(cache_path: Path) -> Optional[pd.DataFrame]:
    """
    Load cached data from parquet file.
    
    Args:
        cache_path: Path to cache file
        
    Returns:
        DataFrame if file exists, None otherwise
    """
    if cache_path.exists():
        try:
            return pd.read_parquet(cache_path)
        except Exception as e:
            print(f"Error loading cache from {cache_path}: {e}")
            return None
    return None


def save_cached_data(data: pd.DataFrame, cache_path: Path) -> None:
    """
    Save data to cache as parquet file.
    
    Args:
        data: DataFrame to save
        cache_path: Path to cache file
    """
    try:
        ensure_cache_dir(cache_path)
        data.to_parquet(cache_path, index=False)
    except Exception as e:
        print(f"Error saving cache to {cache_path}: {e}")


def is_cached(cache_path: Path) -> bool:
    """
    Check if data is cached.
    
    Args:
        cache_path: Path to cache file
        
    Returns:
        True if cache file exists, False otherwise
    """
    return cache_path.exists()

print("✅ Cache Read/Write Functions Defined")


## 3.3 Cache Statistics & Inspection

**Data Flow:**
- **Input**: Optional data_type, symbol filters
- **Processing**: Scan cache directories and collect statistics
- **Output**: DataFrame with cache statistics
- **Dependencies**: Section 3.1 (cache paths)


In [ ]:
def get_cache_stats() -> Dict[str, Any]:
    """
    Get comprehensive cache statistics.
    
    Returns:
        Dict with cache statistics including:
        - data_type_breakdown: Count by type
        - ticker_breakdown: Count by ticker
        - date_ranges: Date ranges per ticker
        - file_counts: Total file counts
        - sizes: Total sizes in MB
    """
    stats = {
        'data_type_breakdown': {},
        'ticker_breakdown': {},
        'date_ranges': {},
        'file_counts': {},
        'sizes': {},
    }
    
    # Scan all cache directories
    for data_type, cache_dir in [
        ('equity', CACHE_EQUITY_DIR),
        ('options', CACHE_OPTIONS_DIR),
        ('processed', CACHE_PROCESSED_DIR),
    ]:
        if cache_dir.exists():
            files = list(cache_dir.glob("*.parquet"))
            stats['data_type_breakdown'][data_type] = len(files)
            stats['file_counts'][data_type] = len(files)
            
            # Calculate total size
            total_size = sum(f.stat().st_size for f in files)
            stats['sizes'][data_type] = total_size / (1024 * 1024)  # MB
            
            # Parse files for ticker and date info
            for f in files:
                parts = f.stem.split('_')
                if len(parts) >= 2:
                    ticker = parts[0]
                    date_str = parts[1]
                    
                    if ticker not in stats['ticker_breakdown']:
                        stats['ticker_breakdown'][ticker] = {}
                        stats['date_ranges'][ticker] = {'min': None, 'max': None}
                    
                    if data_type not in stats['ticker_breakdown'][ticker]:
                        stats['ticker_breakdown'][ticker][data_type] = 0
                    stats['ticker_breakdown'][ticker][data_type] += 1
                    
                    # Update date range
                    try:
                        d = date.fromisoformat(date_str)
                        if stats['date_ranges'][ticker]['min'] is None or d < stats['date_ranges'][ticker]['min']:
                            stats['date_ranges'][ticker]['min'] = d
                        if stats['date_ranges'][ticker]['max'] is None or d > stats['date_ranges'][ticker]['max']:
                            stats['date_ranges'][ticker]['max'] = d
                    except:
                        pass
    
    return stats


def inspect_cache(data_type: Optional[str] = None, symbol: Optional[str] = None) -> pd.DataFrame:
    """
    Inspect cache contents and return as DataFrame.
    
    Args:
        data_type: Optional filter by data type
        symbol: Optional filter by symbol
        
    Returns:
        DataFrame with columns: data_type, symbol, date_range, file_count, total_size_mb
    """
    stats = get_cache_stats()
    rows = []
    
    for ticker, ticker_data in stats['ticker_breakdown'].items():
        if symbol and ticker != symbol:
            continue
        
        for dt, count in ticker_data.items():
            if data_type and dt != data_type:
                continue
            
            date_range = stats['date_ranges'].get(ticker, {})
            date_min = date_range.get('min', 'N/A')
            date_max = date_range.get('max', 'N/A')
            date_range_str = f"{date_min} to {date_max}" if date_min != 'N/A' else 'N/A'
            
            # Calculate size for this ticker/type
            cache_dir = {
                'equity': CACHE_EQUITY_DIR,
                'options': CACHE_OPTIONS_DIR,
                'processed': CACHE_PROCESSED_DIR,
            }.get(dt, None)
            
            size_mb = 0
            if cache_dir:
                files = list(cache_dir.glob(f"{ticker}_*.parquet"))
                size_mb = sum(f.stat().st_size for f in files) / (1024 * 1024)
            
            rows.append({
                'data_type': dt,
                'symbol': ticker,
                'date_range': date_range_str,
                'file_count': count,
                'total_size_mb': round(size_mb, 2),
            })
    
    result_df = pd.DataFrame(rows)
    # Output for inspection
    if not result_df.empty:
        print(f"✅ Cache Inspection Complete")
        print(f"   Total entries: {len(result_df)}")
        print(f"   Total size: {result_df['total_size_mb'].sum():.2f} MB")
        print("\n   Sample:")
        print(result_df.head(10).to_string())
    return result_df

print("✅ Cache Statistics & Inspection Functions Defined")


In [ ]:
def load_from_cache(data_type: str, symbol: str, date_range: List[date]) -> Dict[str, pd.DataFrame]:
    """
    Load cached data for given parameters.
    
    Args:
        data_type: "equity", "options", or "processed"
        symbol: Stock symbol
        date_range: List of dates to load
        
    Returns:
        Dict[str, DataFrame] with date -> DataFrame mapping
    """
    result = {}
    for d in date_range:
        cache_path = get_cache_path(data_type, symbol, d)
        data = load_cached_data(cache_path)
        if data is not None:
            result[d.isoformat()] = data
    return result


def load_all_cached(data_type: str) -> Dict[str, Dict[str, pd.DataFrame]]:
    """
    Load all cached data of a given type.
    
    Args:
        data_type: "equity", "options", or "processed"
        
    Returns:
        Nested dict: symbol -> date -> DataFrame
    """
    cache_dir = {
        'equity': CACHE_EQUITY_DIR,
        'options': CACHE_OPTIONS_DIR,
        'processed': CACHE_PROCESSED_DIR,
    }.get(data_type)
    
    if not cache_dir or not cache_dir.exists():
        return {}
    
    result = {}
    for cache_file in cache_dir.glob("*.parquet"):
        parts = cache_file.stem.split('_')
        if len(parts) >= 2:
            symbol = parts[0]
            date_str = parts[1]
            
            if symbol not in result:
                result[symbol] = {}
            
            data = load_cached_data(cache_file)
            if data is not None:
                result[symbol][date_str] = data
    
    # Output for inspection
    total_files = sum(len(dates) for dates in result.values())
    print(f"✅ Loaded {total_files} cached {data_type} files")
    print(f"   Symbols: {len(result)}")
    return result

print("✅ Cache Loading Functions Defined")


## 3.5 Cache Management

**Data Flow:**
- **Input**: Optional filters (data_type, symbol, date_range)
- **Processing**: Clear cache files
- **Output**: None
- **Dependencies**: Section 3.1 (cache paths)


In [ ]:
def clear_cache(data_type: Optional[str] = None, symbol: Optional[str] = None, 
                date_range: Optional[List[date]] = None) -> None:
    """
    Clear cache files based on filters.
    
    Args:
        data_type: Optional filter by data type
        symbol: Optional filter by symbol
        date_range: Optional filter by date range
    """
    cache_dirs = []
    if data_type:
        cache_dirs = [{
            'equity': CACHE_EQUITY_DIR,
            'options': CACHE_OPTIONS_DIR,
            'processed': CACHE_PROCESSED_DIR,
        }.get(data_type)]
    else:
        cache_dirs = [CACHE_EQUITY_DIR, CACHE_OPTIONS_DIR, CACHE_PROCESSED_DIR]
    
    cleared = 0
    for cache_dir in cache_dirs:
        if cache_dir.exists():
            for cache_file in cache_dir.glob("*.parquet"):
                parts = cache_file.stem.split('_')
                if len(parts) >= 2:
                    file_symbol = parts[0]
                    file_date_str = parts[1]
                    
                    # Apply filters
                    if symbol and file_symbol != symbol:
                        continue
                    
                    if date_range:
                        try:
                            file_date = date.fromisoformat(file_date_str)
                            if file_date not in date_range:
                                continue
                        except:
                            continue
                    
                    # Delete file
                    cache_file.unlink()
                    cleared += 1
    
    print(f"✅ Cleared {cleared} cache files")


def get_cache_size() -> Dict[str, float]:
    """
    Get cache sizes in MB/GB.
    
    Returns:
        Dict with data_type -> size_mb mapping
    """
    stats = get_cache_stats()
    return stats.get('sizes', {})

print("✅ Cache Management Functions Defined")


---

# SECTION 4: Data Processing Functions

## 4.1 Delta Calculation

**Data Flow:**
- **Input**: Spot price, strike, DTE, option type, volatility
- **Processing**: Calculate option delta using Black-Scholes
- **Output**: Delta value and volatility DataFrame
- **Dependencies**: Section 2.2 (equity data), scipy.stats


In [ ]:
def calculate_delta(spot: float, strike: float, dte: int, is_call: bool, 
                   volatility: float = 0.25, risk_free_rate: float = 0.05) -> float:
    """
    Calculate option delta using Black-Scholes formula.
    
    Args:
        spot: Current stock price
        strike: Strike price
        dte: Days to expiration
        is_call: True for call, False for put
        volatility: Annualized volatility (default 0.25 = 25%)
        risk_free_rate: Risk-free rate (default 0.05 = 5%)
        
    Returns:
        Delta value (-1 to 1)
    """
    if dte <= 0 or spot <= 0 or strike <= 0:
        return 0.0
    
    T = dte / 365.0
    try:
        d1 = (math.log(spot / strike) + (risk_free_rate + 0.5 * volatility ** 2) * T) / (volatility * math.sqrt(T))
        if is_call:
            delta = norm.cdf(d1)
        else:
            delta = norm.cdf(d1) - 1
        return round(delta, 4)
    except (ValueError, ZeroDivisionError):
        return 0.0


def estimate_volatility(prices_df: pd.DataFrame) -> float:
    """
    Estimate annualized volatility from historical prices.
    
    Uses VOLATILITY_WINDOW_DAYS (30 days) from strategy parameters.
    
    Args:
        prices_df: DataFrame with 'price' or 'close' column
        
    Returns:
        Annualized volatility (e.g., 0.25 = 25%)
    """
    price_col = 'price' if 'price' in prices_df.columns else 'close'
    window = VOLATILITY_WINDOW_DAYS
    
    if len(prices_df) < window:
        return 0.25  # Default volatility
    
    returns = prices_df[price_col].pct_change().dropna().tail(window)
    if len(returns) == 0:
        return 0.25
    
    # Annualized volatility = std * sqrt(252 trading days)
    daily_std = returns.std()
    annualized_vol = daily_std * math.sqrt(252)
    return max(0.10, min(1.0, annualized_vol))  # Bound between 10% and 100%


def add_delta_to_chain(chain_df: pd.DataFrame, spot_price: float, 
                      equity_history: pd.DataFrame) -> pd.DataFrame:
    """
    Add delta column to options chain.
    
    Args:
        chain_df: DataFrame with option chain
        spot_price: Current spot price
        equity_history: Historical equity prices for volatility estimation
        
    Returns:
        DataFrame with 'delta' and 'estimated_volatility' columns added
    """
    result = chain_df.copy()
    vol = estimate_volatility(equity_history)
    
    deltas = []
    for _, row in result.iterrows():
        is_call = row['type'] == 'call'
        delta = calculate_delta(spot_price, row['strike'], row['dte'], is_call, vol)
        deltas.append(delta)
    
    result['delta'] = deltas
    result['estimated_volatility'] = vol
    
    # Output for inspection
    print(f"✅ Added delta to {len(result)} contracts")
    print(f"   Volatility: {vol:.2%}")
    print(f"   Delta range: {result['delta'].min():.3f} to {result['delta'].max():.3f}")
    return result

print("✅ Delta Calculation Functions Defined")


## 4.2 Technical Indicators

**Data Flow:**
- **Input**: Price DataFrame, window parameters
- **Processing**: Calculate SMA and Bollinger Bands
- **Output**: DataFrame with indicators added
- **Dependencies**: None


In [ ]:
def calculate_sma(prices: pd.Series, window: int) -> pd.Series:
    """
    Calculate Simple Moving Average.
    
    Args:
        prices: Price series
        window: Lookback period
        
    Returns:
        Series with SMA values
    """
    return prices.rolling(window=window, min_periods=window).mean()


def calculate_bollinger_bands(prices: pd.Series, window: int, std_dev: float) -> pd.DataFrame:
    """
    Calculate Bollinger Bands.
    
    Args:
        prices: Price series
        window: Lookback period
        std_dev: Number of standard deviations
        
    Returns:
        DataFrame with columns: middle, upper, lower
    """
    middle = calculate_sma(prices, window)
    std = prices.rolling(window=window, min_periods=window).std()
    upper = middle + std_dev * std
    lower = middle - std_dev * std
    return pd.DataFrame({"middle": middle, "upper": upper, "lower": lower})


def add_indicators(prices_df: pd.DataFrame, sma_window: int, bb_window: int, bb_std: float) -> pd.DataFrame:
    """
    Add technical indicators to price DataFrame.
    
    Args:
        prices_df: DataFrame with 'price' or 'close' column
        sma_window: SMA lookback period
        bb_window: Bollinger Bands lookback period
        bb_std: Bollinger Bands standard deviations
        
    Returns:
        DataFrame with indicators added
    """
    result = prices_df.copy()
    price_col = 'price' if 'price' in result.columns else 'close'
    
    # SMA
    result[f'sma_{sma_window}'] = calculate_sma(result[price_col], sma_window)
    
    # Bollinger Bands
    bb = calculate_bollinger_bands(result[price_col], bb_window, bb_std)
    result['bb_middle'] = bb['middle']
    result['bb_upper'] = bb['upper']
    result['bb_lower'] = bb['lower']
    
    # Output for inspection
    print(f"✅ Added indicators to {len(result)} price points")
    print(f"   Columns: {list(result.columns)}")
    return result

print("✅ Technical Indicators Functions Defined")


## 4.3 Beta Calculation

**Data Flow:**
- **Input**: Stock prices, index prices
- **Processing**: Calculate beta (covariance/variance)
- **Output**: DataFrame with beta and statistics
- **Dependencies**: Section 2.2 (equity data), Section 2.4 (index prices)


In [ ]:
def calculate_beta(stock_prices: pd.DataFrame, index_prices: pd.DataFrame) -> float:
    """
    Calculate beta of a stock relative to the market.
    
    Uses BETA_WINDOW_DAYS (756 days = 3 years) from strategy parameters.
    
    Args:
        stock_prices: DataFrame with 'price' or 'close' column
        index_prices: DataFrame with 'price' or 'close' column (e.g., SPY)
        
    Returns:
        Beta value
    """
    window = BETA_WINDOW_DAYS
    stock_col = 'price' if 'price' in stock_prices.columns else 'close'
    index_col = 'price' if 'price' in index_prices.columns else 'close'
    
    stock_close = stock_prices[stock_col].tail(window)
    index_close = index_prices[index_col].tail(window)
    
    # Align dates
    common_dates = stock_close.index.intersection(index_close.index)
    if len(common_dates) < 20:
        return 1.0  # Default beta
    
    stock_close = stock_close.loc[common_dates]
    index_close = index_close.loc[common_dates]
    
    # Calculate returns
    stock_returns = stock_close.pct_change().dropna()
    index_returns = index_close.pct_change().dropna()
    
    # Align after pct_change
    common_idx = stock_returns.index.intersection(index_returns.index)
    if len(common_idx) < 20:
        return 1.0
    
    stock_returns = stock_returns.loc[common_idx]
    index_returns = index_returns.loc[common_idx]
    
    # Calculate beta
    covariance = np.cov(stock_returns, index_returns)[0, 1]
    variance = np.var(index_returns)
    
    if variance == 0:
        return 1.0
    
    beta = covariance / variance
    return max(0.0, min(3.0, beta))  # Bound to reasonable range


def calculate_betas_for_universe(symbols: List[str], equity_data: Dict[str, pd.DataFrame], 
                                 index_data: pd.DataFrame) -> Dict[str, float]:
    """
    Calculate betas for all symbols in universe.
    
    Args:
        symbols: List of stock symbols
        equity_data: Dict[symbol] -> DataFrame with equity prices
        index_data: DataFrame with index prices
        
    Returns:
        Dict[symbol] -> beta
    """
    betas = {}
    for symbol in symbols:
        if symbol in equity_data:
            beta = calculate_beta(equity_data[symbol], index_data)
            betas[symbol] = beta
    
    # Output for inspection
    beta_df = pd.DataFrame([{'symbol': s, 'beta': betas.get(s, 1.0)} for s in symbols])
    print(f"✅ Calculated betas for {len(betas)} symbols")
    print(f"   Beta range: {beta_df['beta'].min():.3f} to {beta_df['beta'].max():.3f}")
    print(f"   Mean beta: {beta_df['beta'].mean():.3f}")
    return betas

print("✅ Beta Calculation Functions Defined")


---

# SECTION 5: Filtering Functions

## 5.1 Fundamental Filters

**Data Flow:**
- **Input**: Symbols, market caps, betas, prices
- **Processing**: Filter by market cap, beta, price
- **Output**: DataFrame with filter results
- **Dependencies**: Section 2.4 (market data), Section 4.3 (beta)


In [ ]:
def filter_by_market_cap(symbols: List[str], market_caps: Dict[str, float], 
                         min_cap: float, max_cap: Optional[float] = None) -> List[str]:
    """Filter symbols by market cap. Returns filtered list and DataFrame for inspection."""
    filtered = []
    results = []
    for s in symbols:
        cap = market_caps.get(s, 0)
        passes = cap >= min_cap and (max_cap is None or cap <= max_cap)
        results.append({'symbol': s, 'market_cap': cap, 'passed_filter': passes})
        if passes:
            filtered.append(s)
    
    result_df = pd.DataFrame(results)
    print(f"✅ Market cap filter: {len(filtered)}/{len(symbols)} passed")
    return filtered, result_df


def filter_by_beta(symbols: List[str], betas: Dict[str, float], 
                   min_beta: float, max_beta: float) -> List[str]:
    """Filter symbols by beta. Returns filtered list and DataFrame for inspection."""
    filtered = []
    results = []
    for s in symbols:
        beta = betas.get(s, 1.0)
        passes = min_beta <= beta <= max_beta
        results.append({'symbol': s, 'beta': beta, 'passed_filter': passes})
        if passes:
            filtered.append(s)
    
    result_df = pd.DataFrame(results)
    print(f"✅ Beta filter: {len(filtered)}/{len(symbols)} passed")
    return filtered, result_df


def filter_by_price(symbols: List[str], prices: Dict[str, float], 
                   min_price: float, max_price: float) -> List[str]:
    """Filter symbols by price. Returns filtered list and DataFrame for inspection."""
    filtered = []
    results = []
    for s in symbols:
        price = prices.get(s, 0)
        passes = min_price <= price <= max_price
        results.append({'symbol': s, 'price': price, 'passed_filter': passes})
        if passes:
            filtered.append(s)
    
    result_df = pd.DataFrame(results)
    print(f"✅ Price filter: {len(filtered)}/{len(symbols)} passed")
    return filtered, result_df

print("✅ Fundamental Filter Functions Defined")


## 5.2 Technical Filters

**Data Flow:**
- **Input**: Symbols, price data with indicators
- **Processing**: Check if price passes technical filter (SMA/BB)
- **Output**: DataFrame with filter results
- **Dependencies**: Section 4.2 (indicators)


In [ ]:
def check_technical_filter(price: float, sma: float, bb_lower: float, bb_upper: float, 
                          filter_type: str) -> bool:
    """Check if price passes technical filter."""
    if filter_type == "NONE":
        return True
    elif filter_type == "SMA_ONLY":
        return price <= sma
    elif filter_type == "BOLLINGER_ONLY":
        return price <= bb_lower
    elif filter_type == "SMA_OR_BOLLINGER":
        return price <= sma or price <= bb_lower
    elif filter_type == "SMA_AND_BOLLINGER":
        return price <= sma and price <= bb_lower
    return True


def apply_technical_filter(symbols: List[str], price_data: Dict[str, pd.DataFrame], 
                          filter_type: str) -> List[str]:
    """Apply technical filter. Returns filtered list and DataFrame for inspection."""
    filtered = []
    results = []
    
    for s in symbols:
        if s not in price_data or price_data[s].empty:
            continue
        
        df = price_data[s]
        if 'price' not in df.columns or len(df) == 0:
            continue
        
        latest = df.iloc[-1]
        price = latest['price']
        sma_val = latest.get(f'sma_{SMA_WINDOW}', price)
        bb_lower = latest.get('bb_lower', price)
        bb_upper = latest.get('bb_upper', price)
        
        passes = check_technical_filter(price, sma_val, bb_lower, bb_upper, filter_type)
        reason = "Pass" if passes else "Fail"
        
        results.append({
            'symbol': s,
            'date': latest.get('date', 'N/A'),
            'price': price,
            'sma': sma_val,
            'bb_lower': bb_lower,
            'passes_filter': passes,
            'reason': reason,
        })
        
        if passes:
            filtered.append(s)
    
    result_df = pd.DataFrame(results)
    print(f"✅ Technical filter: {len(filtered)}/{len(symbols)} passed")
    return filtered, result_df

print("✅ Technical Filter Functions Defined")


## 5.3 Options Chain Filters

**Data Flow:**
- **Input**: Options chain DataFrame, filter parameters
- **Processing**: Filter by type, DTE, delta, liquidity
- **Output**: Filtered DataFrame with statistics
- **Dependencies**: Section 2.3 (options data), Section 4.1 (delta)


In [ ]:
def filter_by_type(chain_df: pd.DataFrame, opt_type: str) -> pd.DataFrame:
    """Filter chain by option type. Returns filtered DataFrame."""
    if chain_df.empty or 'type' not in chain_df.columns:
        return pd.DataFrame()
    filtered = chain_df[chain_df['type'] == opt_type].copy()
    print(f"   Type filter ({opt_type}): {len(filtered)}/{len(chain_df)} contracts")
    return filtered


def filter_by_dte(chain_df: pd.DataFrame, min_dte: int, max_dte: int) -> pd.DataFrame:
    """Filter chain by DTE. Returns filtered DataFrame."""
    if chain_df.empty or 'dte' not in chain_df.columns:
        return pd.DataFrame()
    filtered = chain_df[(chain_df['dte'] >= min_dte) & (chain_df['dte'] <= max_dte)].copy()
    print(f"   DTE filter ({min_dte}-{max_dte}): {len(filtered)}/{len(chain_df)} contracts")
    if not filtered.empty:
        print(f"      DTE range: {filtered['dte'].min()}-{filtered['dte'].max()}")
    return filtered


def filter_by_delta(chain_df: pd.DataFrame, target_delta: float, delta_band: float, 
                   opt_type: str) -> pd.DataFrame:
    """Filter chain by delta. Returns filtered DataFrame."""
    if chain_df.empty or 'delta' not in chain_df.columns:
        return pd.DataFrame()
    
    delta_min = target_delta - delta_band
    delta_max = target_delta + delta_band
    
    if opt_type == 'put':
        # For puts, use absolute delta
        filtered = chain_df[chain_df['delta'].abs().between(delta_min, delta_max)].copy()
    else:
        filtered = chain_df[chain_df['delta'].between(delta_min, delta_max)].copy()
    
    print(f"   Delta filter ({delta_min:.2f}-{delta_max:.2f}): {len(filtered)}/{len(chain_df)} contracts")
    if not filtered.empty:
        print(f"      Delta range: {filtered['delta'].abs().min():.3f}-{filtered['delta'].abs().max():.3f}")
    return filtered


def filter_by_liquidity(chain_df: pd.DataFrame, min_oi: int, min_volume: int, 
                       max_spread_pct: float) -> pd.DataFrame:
    """Filter chain by liquidity. Returns filtered DataFrame."""
    if chain_df.empty:
        return pd.DataFrame()
    
    filtered = chain_df.copy()
    
    # Open interest filter
    oi_col = 'open_interest' if 'open_interest' in filtered.columns else 'oi'
    if oi_col in filtered.columns:
        filtered = filtered[filtered[oi_col] >= min_oi]
    
    # Volume filter
    if 'volume' in filtered.columns:
        filtered = filtered[filtered['volume'] >= min_volume]
    
    # Spread filter
    if 'bid' in filtered.columns and 'ask' in filtered.columns:
        mid_price = (filtered['bid'] + filtered['ask']) / 2
        spread = filtered['ask'] - filtered['bid']
        spread_pct = spread / mid_price
        filtered = filtered[spread_pct <= max_spread_pct]
    
    print(f"   Liquidity filter: {len(filtered)}/{len(chain_df)} contracts")
    return filtered


def filter_chain_for_wheel(chain_df: pd.DataFrame, opt_type: str, config: Dict, 
                           as_of_date: date) -> pd.DataFrame:
    """Apply all filters for wheel strategy. Returns filtered DataFrame with breakdown."""
    if chain_df.empty:
        return pd.DataFrame()
    
    print(f"\n🔍 Filtering chain for {opt_type} on {as_of_date}")
    print(f"   Initial contracts: {len(chain_df)}")
    
    # Apply filters in sequence
    filtered = filter_by_type(chain_df, opt_type)
    filtered = filter_by_dte(filtered, config['min_dte'], config['max_dte'])
    filtered = filter_by_delta(filtered, config['delta_target'], config['delta_band'], opt_type)
    filtered = filter_by_liquidity(filtered, config['min_oi'], config['min_volume'], 
                                   config['max_spread_pct'])
    
    print(f"   ✅ Final filtered: {len(filtered)} contracts")
    return filtered

print("✅ Options Chain Filter Functions Defined")


---

# SECTION 6-9: Strategy Logic, Backtest Engine, Reporting, and Main Execution

**Note**: Due to notebook size, Sections 6-9 contain placeholder functions. 
These should be implemented following the same pattern as Sections 1-5:
- Small, single-purpose functions
- Clear inputs/outputs
- DataFrame/dict outputs for inspection
- Comprehensive docstrings

**Key Functions Needed:**
- Section 6: Position management, contract selection, rolling logic
- Section 7: Portfolio state, trade execution, daily processing, main backtest loop
- Section 8: Metrics calculation, visualization, reporting
- Section 9: Cache inspection, data collection, pre-computation, backtest execution, results

**Implementation Pattern:**
```python
def function_name(inputs) -> outputs:
    \"\"\"Docstring explaining function.\"\"\"
    # Implementation
    result_df = pd.DataFrame(...)  # For inspection
    print(f"✅ Function complete: {stats}")
    return result, result_df
```

**Next Steps:**
1. Implement Sections 6-9 following the established patterns
2. Test each function independently
3. Integrate into main execution flow
4. Add comprehensive error handling
5. Add progress bars for long-running operations
